# Dataquest Guided Project: Profitable App Profiles for the App Store and Google Play Markets

The goal of this project is to analyize Android and IOS mobile apps. The company we represent builds **free** apps directed towards an **English-speaking** audience, with revenue coming from in-app advertising.

The goal of the project is to learn which types of applications are most likely to attract more users.

## About the Data

* [A data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

### Import the Data

In [1]:
import tarfile

# Extracts the .tar file containing both datasets
app_data_tgz = tarfile.open('Guided Project_ Profitable App Profiles for the App Store and Google Play Markets.tar')
app_data_tgz.extractall()
app_data_tgz.close()

Open the two data sets and save them to list variables:

In [2]:
def open_csv_dataset(file, header=True):
    """Converts a CSV file into a Python list for analysis.
     Parameters:
        - file: file name
        - header=True: True if first row of data represents categories.
        
    Returns:
        - header, list (if header=True)
        - list (if header=False)
        """
    from csv import reader
    open_file = open(file, encoding='utf-8')
    read_file = reader(open_file)
    name = list(read_file)
    open_file.close()
    
    if header:
        col_names = name[0]
        return col_names, name[1:]
    else:
        return name

In [3]:
IOS_COLS, IOS_DATA = open_csv_dataset('Applestore.csv')
ANDROID_COLS, ANDROID_DATA = open_csv_dataset('googleplaystore.csv')

In [4]:
help(open_csv_dataset)

Help on function open_csv_dataset in module __main__:

open_csv_dataset(file, header=True)
    Converts a CSV file into a Python list for analysis.
     Parameters:
        - file: file name
        - header=True: True if first row of data represents categories.
        
    Returns:
        - header, list (if header=True)
        - list (if header=False)



In [5]:
def explore_data(dataset, start, end, rows_and_columns=False):
    """Function to explore a dataset based on the specified indices
    
    Parameters:
        - dataset: variable representing the dataset
        - start: start index within dataset
        - end: stop index within dataset
        - rows_and_columns: prints the number of rows and columns, if True
        
    Returns:
        None
        """
    
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # Adds a new line after each row
        
    if rows_and_columns:
        print(f'Number of rows: {len(dataset)}')
        print(f'Number of columns: {len(dataset[0])}')

### Explore Datasets

In [6]:
explore_data(IOS_DATA, 3, 5, rows_and_columns=True)

['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 7197
Number of columns: 16


In [7]:
explore_data(ANDROID_DATA, 3, 5, rows_and_columns=True)

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 10841
Number of columns: 13


### Explore Column Names

In [8]:
# IOS Columns:
for col in IOS_COLS:
    print(col)

id
track_name
size_bytes
currency
price
rating_count_tot
rating_count_ver
user_rating
user_rating_ver
ver
cont_rating
prime_genre
sup_devices.num
ipadSc_urls.num
lang.num
vpp_lic


Of these IOS columns, the following might be useful for our purpose:
* price
* user_rating
* cont_rating (Maybe)
* prime genre
* sup_devices.num (Maybe)
* lang.num (Maybe)

[Link to data](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [9]:
# Android Columns:
for col in ANDROID_COLS:
    print(col)

App
Category
Rating
Reviews
Size
Installs
Type
Price
Content Rating
Genres
Last Updated
Current Ver
Android Ver


Of these Android columns, the following might be useful:
* Category
* Rating
* Reviews
* Size
* Installs
* Type
* Content Rating
* Genres

Price should be superfluous, as we will hone in on type=Free.

### Data Cleaning

Data cleaning is done before the analysis; it includes removing or correcting wrong data, removing duplicate data, and modifying the data to fit the purpose of our analysis.

**To-Do:**
1. Identify the issue with a well-known error in one of the Android rows.
    * [Discussion Source](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015)
    
Data:
* IOS_COLS, IOS_DATA 
* ANDROID_COLS, ANDROID_DATA

In [10]:
# Labels:
print(ANDROID_COLS)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [11]:
def error_checker(dataset, dataset_header=None):
    """Checks for length inconsistencies within the dataset.
    
    Parameters:
        - dataset_header: Header (labels) of the data, if available
        - dataset: dataset (list)
        
    Returns:
        - error_list: nested list of errors. Includes:
            * error[0]: Data values (to help with inspection)
            * error[1]: Index of the error within the dataset
    """
    error_list = []
    if not dataset_header:
        dataset_header = dataset[0]
        
    for row in dataset:
        if len(row) != len(dataset_header):
            error = []
            idx = dataset.index(row)
            error.append(row)
            error.append(idx)
            error_list.append(error)
            print(f'Data error found at index: {idx}')
    num_errors = len(error_list)
    print(f'{num_errors} length inconsistencies in this dataset')
    return error_list

In [12]:
# for row in ANDROID_DATA:
#     if len(row) != len(ANDROID_COLS):
#         print(row)
#         print(f'The index is: {ANDROID_DATA.index(row)}')

error_checker(ANDROID_DATA, ANDROID_COLS)

Data error found at index: 10472
1 length inconsistencies in this dataset


[[['Life Made WI-Fi Touchscreen Photo Frame',
   '1.9',
   '19',
   '3.0M',
   '1,000+',
   'Free',
   '0',
   'Everyone',
   '',
   'February 11, 2018',
   '1.0.19',
   '4.0 and up'],
  10472]]

#### Findings:
* "Genres" is missing from index 10472 in the ANDROID_Data. I have decided to delete this row from the dataset, rather than augment it.

In [13]:
ANDROID_DATA[10472]

['Life Made WI-Fi Touchscreen Photo Frame',
 '1.9',
 '19',
 '3.0M',
 '1,000+',
 'Free',
 '0',
 'Everyone',
 '',
 'February 11, 2018',
 '1.0.19',
 '4.0 and up']

In [14]:
del(ANDROID_DATA[10472])

In [15]:
android_errors = error_checker(ANDROID_DATA, ANDROID_COLS)

0 length inconsistencies in this dataset


In [16]:
ios_errors = error_checker(IOS_DATA, IOS_COLS)

0 length inconsistencies in this dataset


### Check for Redundant Apps

In [17]:
def find_duplicates(dataset, attr_idx):
    """Checks a dataset list for multiple occurrences of a row
    
    Parameters:
        - dataset: (list) Dataset, excluding headers
        - attr_idx (int) Index of the data attribute
        
    Returns:
        - num_duplicates (int): Number of duplicate entries within the dataset.
        - duplicate_apps (list): List of duplicate apps
    
    """
    duplicate_apps = []
    unique_apps = []
    
    for row in dataset:
        name = row[0]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    
    num_duplicates = len(duplicate_apps)
    print(f'{num_duplicates} duplicates found')
    
    return num_duplicates, duplicate_apps

In [18]:
android_num_duplicates, android_duplicates = find_duplicates(ANDROID_DATA, 0)

1181 duplicates found


In [19]:
ios_num_duplicates, ios_duplicates = find_duplicates(IOS_DATA, 0)

0 duplicates found


As shown above, the `ANDROID_DATA` has 1181 duplicates, while the `IOS_DATA` has none. 

Within these duplicates, all fields are the same except for the number of reviews (index 3). My strategy will be to keep the entry with the highest number of reviews, discarding the others. This should be reasonable, as the remaining entry will be the most frequently-updated representation of the app.

In [20]:
print(ios_num_duplicates)

0


### Remove Duplicate Android Entries

Verify the target length of a unique dictionary for ANDROID_DATA:

In [21]:
print('Number of apps:', len(ANDROID_DATA) - 1181)

Number of apps: 9659


**Target length is 9659**

In [22]:
# Explore index/columns within ANDROID_DATA:
print(list(enumerate(ANDROID_COLS)))

[(0, 'App'), (1, 'Category'), (2, 'Rating'), (3, 'Reviews'), (4, 'Size'), (5, 'Installs'), (6, 'Type'), (7, 'Price'), (8, 'Content Rating'), (9, 'Genres'), (10, 'Last Updated'), (11, 'Current Ver'), (12, 'Android Ver')]


I will now: 
* Create a blank dictionary, `reviews_max`
* Iterate through ANDROID_DATA, assigning variables for `name` and `n_reviews`
* If the app is already included in `reviews_max` and the number of reviews is greater than the existing value, the `n_reviews` value is increased accordingly.
* If the app is **not** already in `reviews_max`, it is added with the corresponding `n_reviews` value.
*Note: If the app is already included in the dictionary but it has a lower n_reviews value, it is ignored. The goal is to populate the dictionary with each app having the most reviews.

In [23]:
reviews_max = dict()
for app in ANDROID_DATA:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and n_reviews > reviews_max[name]:
        reviews_max[name] = n_reviews
    if name not in reviews_max:   # "else" statement wouldn't account for existing entries with HIGHER reviews
        reviews_max[name] = n_reviews

In [24]:
len(reviews_max)

9659

Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:

* We start by initializing two empty lists, android_clean and already_added.
* We loop through the android data set, and for every iteration:
    * We isolate the name of the app and the number of reviews (as a float).
    * We add the current row (app) to the android_clean list, and the app name (name) to the already_added list if:
        * The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
        * The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

In [49]:
android_clean = []
already_added = []

for app in ANDROID_DATA:
    name = app[0]
    n_reviews = float(app[3])
    
    if ( n_reviews == reviews_max[name] ) and ( name not in already_added ):
        android_clean.append(app)
        already_added.append(name)

In [50]:
len(android_clean)

9659

IOS_DATA is already "clean" insofar as not having any duplicates. For consistency, I will also update that dataset to `ios_clean`:

In [51]:
ios_clean = IOS_DATA

In [52]:
len(android_clean), len(ios_clean)

(9659, 7197)

### Remove Non-English Apps

Our goal is to only analyize free, *english* applications.

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

#### ord(str) and chr(int)

In [25]:
ord('a')

97

In [26]:
chr(497)

'Ǳ'

#### Create a function to detect non-english characters

In [27]:
def is_english(string):
    for c in string:
        if ord(c) > 127:
            return False
    return True

Test function:

In [28]:
test_1 = '爱奇艺PPS -《欢乐颂2》电视剧热播'
test_2 = 'Docs To Go™ Free Office Suite'
test_3 = 'Instachat 😜'
test_4 = 'Instagram'

In [29]:
test_list = [test_1, test_2, test_3, test_4]

for test in test_list:
    print(is_english(test))

False
False
False
True


To minimize the impact of data loss, we'll only remove an app if its name has more than three characters outside of ASCII range 0-127

#### Redefine `is_english` to Filter Only Strings with More Than Three Abnormal ASCII Characters

In [30]:
def is_english(string):
    num_abnormal_ascii = 0
    for c in string:
        if ord(c) > 127:
            num_abnormal_ascii += 1
    return False if num_abnormal_ascii > 3 else True

In [31]:
is_english('Docs To Go™ Free Office Suite')

True

In [32]:
is_english('Instachat 😜😜')

True

In [33]:
is_english('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

As a reminder, the current datasets that we are working with are as follows:
* `android_clean`
* `ios_clean`

Now, let's use our new `is_english` function to filter non-english apps.

In [45]:
def filter_only_english(dataset, idx):
    """Accepts a list and returns a list of english-only applications"""
    
    english_apps = []
    for app in dataset:
        name = app[idx]
        if is_english(name):
            english_apps.append(app)
    
    return english_apps

In [53]:
android_english = filter_only_english(android_clean, 0)
ios_english = filter_only_english(ios_clean, 1)

Compare Original Datasets to Filtered Datasets:

In [55]:
len(android_clean), len(android_english)

(9659, 9614)

In [56]:
len(ios_clean), len(ios_english)

(7197, 6183)

So far in the data cleaning process, we:

* Removed inaccurate data
* Removed duplicate app entries
* Removed non-English apps

We only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps; we'll need to isolate only the free apps for our analysis.

Isolating the free apps will be our last step in the data cleaning process.

### Isolating Free Apps

In [61]:
print(list(enumerate(ANDROID_COLS)))

[(0, 'App'), (1, 'Category'), (2, 'Rating'), (3, 'Reviews'), (4, 'Size'), (5, 'Installs'), (6, 'Type'), (7, 'Price'), (8, 'Content Rating'), (9, 'Genres'), (10, 'Last Updated'), (11, 'Current Ver'), (12, 'Android Ver')]


For android, free apps are identified by **index 7** having a value of '0.0'.
*Note: I originally filtered this if index 6 == 'Free', which led to a single app that didn't satisfy both conditions (shown below).

In [62]:
print(list(enumerate(IOS_COLS)))

[(0, 'id'), (1, 'track_name'), (2, 'size_bytes'), (3, 'currency'), (4, 'price'), (5, 'rating_count_tot'), (6, 'rating_count_ver'), (7, 'user_rating'), (8, 'user_rating_ver'), (9, 'ver'), (10, 'cont_rating'), (11, 'prime_genre'), (12, 'sup_devices.num'), (13, 'ipadSc_urls.num'), (14, 'lang.num'), (15, 'vpp_lic')]


For ios, Price is located at **index 4**

So, we will explore apps that have:
* android_english[7] == '0'
* ios_english[4] == '0.0'

In [77]:
ios_english[0]

['284882215',
 'Facebook',
 '389879808',
 'USD',
 '0.0',
 '2974676',
 '212',
 '3.5',
 '3.5',
 '95.0',
 '4+',
 'Social Networking',
 '37',
 '1',
 '29',
 '1']

In [81]:
android_free = []
ios_free = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_free.append(app)
        
for app in ios_english:
    name = app[1]
    price = app[4]
    if price == '0.0':
        ios_free.append(app)

**Android, before and after:**

In [82]:
len(android_english), len(android_free)

(9614, 8864)

In [80]:
len(ios_english), len(ios_free)

(6183, 3222)

#### Interesting Finding:

When Checking my work, I realized I should have 8864 android apps remaining. I originally had 8863; 1 less. I decided to try to find the app that had a price of '0' but did not have the type = 'Free'. It turned out to be *Command & Conquer: Rivals*:

In [83]:
android_strange = []

for app in android_english:
    type_app = app[6]
    price = app[7]
    if price == '0' and type_app != 'Free':
        android_strange.append(app)

In [84]:
len(android_strange)

1

In [85]:
android_strange

[['Command & Conquer: Rivals',
  'FAMILY',
  'NaN',
  '0',
  'Varies with device',
  '0',
  'NaN',
  '0',
  'Everyone 10+',
  'Strategy',
  'June 28, 2018',
  'Varies with device',
  'Varies with device']]

## Data Analysis: Most Common Apps by Genre

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.
Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our data sets.

### Explore Android Data
Current dataset is `android_free`. For Android, category (index 1) and genres (index 9) appear to be representative of the type of app. 

In [90]:
print(list(enumerate(ANDROID_COLS)))

[(0, 'App'), (1, 'Category'), (2, 'Rating'), (3, 'Reviews'), (4, 'Size'), (5, 'Installs'), (6, 'Type'), (7, 'Price'), (8, 'Content Rating'), (9, 'Genres'), (10, 'Last Updated'), (11, 'Current Ver'), (12, 'Android Ver')]


### Explore IOS Data
Current dataset is `ios_free`. For ios, prime_genre (index 11) appears to be the best representation of the type of app.

In [98]:
print(list(enumerate(IOS_COLS)))

[(0, 'id'), (1, 'track_name'), (2, 'size_bytes'), (3, 'currency'), (4, 'price'), (5, 'rating_count_tot'), (6, 'rating_count_ver'), (7, 'user_rating'), (8, 'user_rating_ver'), (9, 'ver'), (10, 'cont_rating'), (11, 'prime_genre'), (12, 'sup_devices.num'), (13, 'ipadSc_urls.num'), (14, 'lang.num'), (15, 'vpp_lic')]


### Frequency Exploration Functions

Let's start by building functions that can generate frequency table showing percentage of the overall dataset.

In [91]:
def freq_table(dataset, index):
    """Accepts a dataset (list) and desired category (value), returning a frequency table for the category"""
    table = dict()
    total = 0
    
    for row in dataset:
        total +=1 
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
            
    table_percentages = dict()
    for key in table:
        percentage = ( table[key] / total ) * 100
        table_percentages[key] = percentage
        
    return table_percentages

In [95]:
test = freq_table(android_free, 1)

In [94]:
total = 0
for value in test.values():
    total += value
print(f'The total percentage is: {total}')

The total percentage is: 100.0


In [96]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key) # value comes first to ensure sorted function works correctly.
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(f'{entry[1]}: {entry[0]}')

### Use Functions to Explore Datasets

#### Android Data
Exploring category (index 1) and genres (index 9):

In [99]:
android_category = display_table(android_free, 1)

FAMILY: 18.907942238267147
GAME: 9.724729241877256
TOOLS: 8.461191335740072
BUSINESS: 4.591606498194946
LIFESTYLE: 3.9034296028880866
PRODUCTIVITY: 3.892148014440433
FINANCE: 3.7003610108303246
MEDICAL: 3.531137184115524
SPORTS: 3.395758122743682
PERSONALIZATION: 3.3167870036101084
COMMUNICATION: 3.2378158844765346
HEALTH_AND_FITNESS: 3.0798736462093865
PHOTOGRAPHY: 2.944494584837545
NEWS_AND_MAGAZINES: 2.7978339350180503
SOCIAL: 2.6624548736462095
TRAVEL_AND_LOCAL: 2.33528880866426
SHOPPING: 2.2450361010830324
BOOKS_AND_REFERENCE: 2.1435018050541514
DATING: 1.861462093862816
VIDEO_PLAYERS: 1.7937725631768955
MAPS_AND_NAVIGATION: 1.3989169675090252
FOOD_AND_DRINK: 1.2409747292418771
EDUCATION: 1.1620036101083033
ENTERTAINMENT: 0.9589350180505415
LIBRARIES_AND_DEMO: 0.9363718411552346
AUTO_AND_VEHICLES: 0.9250902527075812
HOUSE_AND_HOME: 0.8235559566787004
WEATHER: 0.8009927797833934
EVENTS: 0.7107400722021661
PARENTING: 0.6543321299638989
ART_AND_DESIGN: 0.6430505415162455
COMICS: 0.62

In [100]:
android_genres = display_table(android_free, 9)

Tools: 8.449909747292418
Entertainment: 6.069494584837545
Education: 5.347472924187725
Business: 4.591606498194946
Productivity: 3.892148014440433
Lifestyle: 3.892148014440433
Finance: 3.7003610108303246
Medical: 3.531137184115524
Sports: 3.463447653429603
Personalization: 3.3167870036101084
Communication: 3.2378158844765346
Action: 3.1024368231046933
Health & Fitness: 3.0798736462093865
Photography: 2.944494584837545
News & Magazines: 2.7978339350180503
Social: 2.6624548736462095
Travel & Local: 2.3240072202166067
Shopping: 2.2450361010830324
Books & Reference: 2.1435018050541514
Simulation: 2.0419675090252705
Dating: 1.861462093862816
Arcade: 1.8501805054151623
Video Players & Editors: 1.7712093862815883
Casual: 1.7599277978339352
Maps & Navigation: 1.3989169675090252
Food & Drink: 1.2409747292418771
Puzzle: 1.128158844765343
Racing: 0.9927797833935018
Role Playing: 0.9363718411552346
Libraries & Demo: 0.9363718411552346
Auto & Vehicles: 0.9250902527075812
Strategy: 0.913808664259927

*Tools* ranks high on both lists

#### IOS Data
Exploring prime_genre (index 11):

In [101]:
ios_prime_genre = display_table(ios_free, 11)

Games: 58.16263190564867
Entertainment: 7.883302296710118
Photo & Video: 4.9658597144630665
Education: 3.662321539416512
Social Networking: 3.2898820608317814
Shopping: 2.60707635009311
Utilities: 2.5139664804469275
Sports: 2.1415270018621975
Music: 2.0484171322160147
Health & Fitness: 2.0173805090006205
Productivity: 1.7380509000620732
Lifestyle: 1.5828677839851024
News: 1.3345747982619491
Travel: 1.2414649286157666
Finance: 1.1173184357541899
Weather: 0.8690254500310366
Food & Drink: 0.8069522036002483
Reference: 0.5586592178770949
Business: 0.5276225946617008
Book: 0.4345127250155183
Navigation: 0.186219739292365
Medical: 0.186219739292365
Catalogs: 0.12414649286157665


After comparing both datasets, games/entertainment theme seems to be a commen trend between the two datasets.